# Agentic RAG

In [1]:
# %pip install -r "../requirements.txt"

In [6]:
from dotenv import load_dotenv

load_dotenv()

from knowledge_base import retriever
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain.tools import tool, ToolRuntime
from langchain.agents import create_agent, AgentState
from langchain_core.documents.base import Document


## Tool definition

In [7]:
class CustomAgentState(AgentState):
    user_preferences: dict 

In [ ]:
@tool
def get_to_know_the_user(runtime: ToolRuntime) -> str:
    """Use to get to know user and opening a conversation with the user"""
    user_preferences: dict = runtime.state['user_preferences']
    return f'User prefers the following: {user_preferences}'

In [4]:
@tool
def get_context(query: str) -> tuple[str, list[Document]]:
    """Retrieve information to help answer a query."""
    retrieved_docs = retriever.invoke(query)
    serialized = "\n\n".join(
        (f"Source: {doc.metadata}\nContent: {doc.page_content}")
        for doc in retrieved_docs
    )
    return serialized, retrieved_docs

## Agent Prompt

In [45]:
prompt = """
You are a helpful Clerical assistant. You have access to the following tool:
`get_to_know_the_user`: This is used to get to know the user preferences and assist the user accordingly.
`get_context`: This can be used if you need to find any file information for the user's question. 
It takes in a search parameter 'q' as input and this is used to perform vector search in the vector database having file information.
The tool returns a tuple containing information text having content, file name and document metadata information.

< Answer >
Always remember get to know the user for their preferences using `get_to_know_the_user` tool. Preferences might be anything like answer format, response tone, response length, etc.
Respond to the user question in a friendly manner. If its a general question and if you know the answer, feel free to answer and insist the user that you can help with clerical information.
In case you have used the `get_context` tool and it was helpful, let the user know you have searched for the files and found the most appropriate information in the file 'foo'
</Answer>
"""

### Memory saver for Short term memory

In [41]:
from langgraph.checkpoint.memory import InMemorySaver
cp = InMemorySaver()

## Agent creation

In [47]:
agent = create_agent(
    model=ChatGoogleGenerativeAI(model="gemini-2.5-flash-lite"),
    tools=[get_to_know_the_user, get_context],
    system_prompt=prompt,
    state_schema=CustomAgentState,
    checkpointer=cp
 )

In [48]:
def ask_ai(q, thread_id, **preferences):
    print(preferences)
    res = agent.invoke(
    {
        "messages": [
            {"role": "user", "content": q}
        ],
        "user_preferences": {
            **preferences
        }
    },
    config={
        'configurable': {'thread_id': thread_id}
    }
    )

    for msg in res['messages']:
        msg.pretty_print()

In [49]:
ask_ai(q='hi, get me the customers for bluray', thread_id=1, answer_format='detailed list', talk_like='pirate')

{'answer_format': 'detailed list', 'talk_like': 'pirate'}
================================ Human Message =================================

hi, get me the customers for bluray
================================== Ai Message ==================================
Tool Calls:
  get_context (6c2e197c-3029-4e44-a389-24a89ed4ceba)
 Call ID: 6c2e197c-3029-4e44-a389-24a89ed4ceba
  Args:
    query: customers for bluray
================================= Tool Message =================================
Name: get_context

("Source: {'page': 0, 'title': 'Document', 'source': 'docs/invoice_Aaron Hawkins_40100.pdf', 'creator': 'wkhtmltopdf 0.12.6', 'producer': 'Qt 4.8.7', 'page_label': '1', 'total_pages': 1, 'creationdate': '2023-06-05T19:21:16+00:00'}\nContent: INVOICE\n# 40100\nSuperStore\nBill To\n:\nAaron Hawkins\nShip To\n:\n10035, New York City,\nNew York, United\nStates\nDec 30 2012\nFirst Class\n$112.53\nDate\n:\nShip Mode\n:\nBalance Due\n:\nItem\nQuantity\nRate\nAmount\nVerbatim 25 GB 6x Blu-ray S

In [50]:
ask_ai('what else did he purchase', thread_id=1, answer_format='markdown table')

{'answer_format': 'markdown table'}
================================ Human Message =================================

hi, get me the customers for bluray
================================== Ai Message ==================================
Tool Calls:
  get_context (6c2e197c-3029-4e44-a389-24a89ed4ceba)
 Call ID: 6c2e197c-3029-4e44-a389-24a89ed4ceba
  Args:
    query: customers for bluray
================================= Tool Message =================================
Name: get_context

("Source: {'page': 0, 'title': 'Document', 'source': 'docs/invoice_Aaron Hawkins_40100.pdf', 'creator': 'wkhtmltopdf 0.12.6', 'producer': 'Qt 4.8.7', 'page_label': '1', 'total_pages': 1, 'creationdate': '2023-06-05T19:21:16+00:00'}\nContent: INVOICE\n# 40100\nSuperStore\nBill To\n:\nAaron Hawkins\nShip To\n:\n10035, New York City,\nNew York, United\nStates\nDec 30 2012\nFirst Class\n$112.53\nDate\n:\nShip Mode\n:\nBalance Due\n:\nItem\nQuantity\nRate\nAmount\nVerbatim 25 GB 6x Blu-ray Single Layer Recordable